In [230]:
# Template Notebook for Indvidualized Participant Engagement Classifcation NN Model

# ToDo/Goals: https://docs.google.com/document/d/1_dTnbdRoBV0A7xCxVEyLBdckrlsUnMnRFbbRn39tiXk/edit?usp=sharing
# Data: https://drive.google.com/drive/folders/19aJUAlkTMz7PcZE1q4hogFkjVtwYGcMT
# Upload code to help-seeking github: https://github.com/interaction-lab/help_seeking
# Record model results here: https://docs.google.com/spreadsheets/d/16ye54fSSEuAuDL_j56UIeDB-rIIrxq_kPbtyPRQOrVI/edit?usp=sharing

In [231]:
# Imports

import pandas as pd
import numpy as np
from pandas import *
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn import metrics

from keras.layers import Dense, Activation
from keras.models import Sequential
from keras.utils import np_utils

#### TODO: Specifiy Data Path Based on Participant/Your Computer

In [232]:
# TODO: Specify Data Path based on participant/your computer
# USE data from FS4 folder

file = '../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Help-Seeking/Data/FS4/p7_data_FS4.csv'
data = pd.read_csv(file)

In [233]:
data.head()

,of_confidence,of_success,of_gaze_0_x,of_gaze_0_y,of_gaze_0_z,of_gaze_1_x,of_gaze_1_y,of_gaze_1_z,of_gaze_angle_x,of_gaze_angle_y,...,ts_attempt,skill_NC,skill_OS,skill_EM,diff_1,diff_2,diff_3,diff_4,diff_5,no_game
0,1.0,1.0,0.802148,0.562868,0.109746,0.394162,0.568244,0.018943,0.606309,0.508006,...,0.0,0,1,0,0,1,0,0,0,0
1,1.0,1.0,0.805980,0.553090,0.111663,0.395043,0.564921,0.018227,0.607571,0.505478,...,0.0,0,1,0,0,1,0,0,0,0
2,1.0,1.0,0.802828,0.542876,0.108154,0.413679,0.556325,0.012879,0.610515,0.501938,...,0.0,0,1,0,0,1,0,0,0,0
3,1.0,1.0,0.806615,0.533710,0.110562,0.390660,0.568724,0.019901,0.606730,0.502613,...,0.0,0,1,0,0,1,0,0,0,0
4,1.0,1.0,0.805455,0.529496,0.109395,0.397752,0.560606,0.016963,0.607781,0.500253,...,0.0,0,1,0,0,1,0,0,0,0


In [234]:
# Label Analysis

print(data['engagement'].value_counts())
print()

print(1,data['engagement'].value_counts()[1]/sum(data['engagement'].value_counts()))
print(0,data['engagement'].value_counts()[0]/sum(data['engagement'].value_counts()))

1.0    238113
0.0    155633
Name: engagement, dtype: int64

1 0.6047375719372388
0 0.3952624280627613


### Train Test Split (for chosen Feature Set)

#### TODO: Specify which train-test split you want

1. Random Shuffle Split: 70-30, 50-50, 30-70, 10-90
2. Chronological Split: 70-30, 50-50, 30-70
3. Session Split: 70-30, 50-50, 30-70, 10-90

In [235]:
# Session Split: Train on Percentage of Data From Each Session, Sorted Chronologically 

def session_split(data, size):
    sessions = data['session_num'].unique()
    
    first = True
        
    for s in sessions:
        curr = data.loc[data['session_num'] == s]
        curr = curr.sort_values(['timestamp'], ascending=[True])

        y = curr['engagement']
        X = curr.drop(columns=['timestamp', 'session_num', 'engagement'])

        if first:
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=size, shuffle=True)
            first = False
            
        else:
            CX_train, CX_test, Cy_train, Cy_test = train_test_split(X, y, test_size=size, shuffle=True)
                        
            X_train = X_train.append(CX_train)
            X_test = X_test.append(CX_test)
            y_train = y_train.append(Cy_train)
            y_test = y_test.append(Cy_test)
    
    return X_train, X_test, y_train, y_test

In [236]:
# Sort data (don't need for Random Shuffle, but ok to leave it)

data = data.sort_values(['session_num', 'timestamp'], ascending=[True, True])

In [237]:
# TODO: Uncomment train-test split to run below 

y = data['engagement']
X = data.drop(columns=['timestamp', 'session_num', 'engagement'])

# Choose one of these 


# 1. Random Shuffle Split 50-50
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

### (Extra)
# Random Shuffle Split 70-30
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Random Shuffle Split 30-70
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=42)

# Random Shuffle Split 10-90
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=42)
### 

#############################

# 2. Chronological Split 70-30
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)

# 3. Chronological Split 50-50
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, shuffle=False)

# 4. Chronological Split 30-70
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, shuffle=False)

############################# 

# 5. Session Split 70-30
# X_train, X_test, y_train, y_test = session_split(data, 0.3)

# 6. Session Split 50-50
# X_train, X_test, y_train, y_test = session_split(data, 0.5)

# 7. Session Split 30-70
# X_train, X_test, y_train, y_test = session_split(data, 0.7)

# 8. Session Split 10-90
# X_train, X_test, y_train, y_test = session_split(data, 0.9)

In [238]:
# Sort X,y Train dataframes together after split 

X_train, bogus_test, y_train, bogus_test = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)
X_train, bogus_test, y_train, bogus_test = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)
X_train, bogus_test, y_train, bogus_test = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)
X_train, bogus_test, y_train, bogus_test = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)
X_train, bogus_test, y_train, bogus_test = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)

### Step 3: Neural Network Model

#### TODO: May need to adjust following if poor accurracy
1. Vary num_epochs, batch size
2. Vary between 2, 3, 4 hidden layers
3. Vary amount of nodes in each layer

In [239]:
num_epochs = 1       # if bad, try 1 - 20
size_batch = 256      # if bad, try 16, 32, 64, 128, 256 

In [240]:
# NN Model for FS4 (all OF/OP)

model = Sequential()

# model.add(Dense(units=16, input_dim=(X_train.shape[1]), kernel_initializer='glorot_normal'))
model.add(Dense(units=64, input_dim=(X_train.shape[1]), kernel_initializer='glorot_normal'))
model.add(Activation('relu'))

#model.add(Dense(units=16, kernel_initializer='glorot_normal'))
model.add(Dense(units=32, kernel_initializer='glorot_normal'))
model.add(Activation('relu'))

#model.add(Dense(units=64, kernel_initializer='glorot_normal'))
model.add(Dense(units=16, kernel_initializer='glorot_normal'))
model.add(Activation('relu'))

#model.add(Dense(units=4, kernel_initializer='glorot_normal'))
#model.add(Activation('relu'))

model.add(Dense(units=1))
model.add(Activation('sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_89 (Dense)             (None, 64)                1920      
_________________________________________________________________
activation_89 (Activation)   (None, 64)                0         
_________________________________________________________________
dense_90 (Dense)             (None, 32)                2080      
_________________________________________________________________
activation_90 (Activation)   (None, 32)                0         
_________________________________________________________________
dense_91 (Dense)             (None, 16)                528       
_________________________________________________________________
activation_91 (Activation)   (None, 16)                0         
_________________________________________________________________
dense_92 (Dense)             (None, 1)                 17        
__________

In [241]:
model.compile(loss='binary_crossentropy',  
              optimizer='Adam',
              metrics=['accuracy'])

### 4. Model Evaluation

In [242]:
# use validation split to monitor training, don't use for final recorded accurracy 
# model.fit(X_train, y_train, epochs=num_epochs, batch_size=size_batch, verbose=1, validation_split=0.1)

model.fit(X_train, y_train, epochs=num_epochs, batch_size=size_batch, verbose=1)

Epoch 1/1
118123/118123 [==============================] - 3s 22us/step - loss: 0.2805 - acc: 0.8809


In [243]:
loss_and_metrics = model.evaluate(X_test, y_test)
print(loss_and_metrics)

275623/275623 [==============================] - 14s 51us/step
[1.2370253477332234, 0.650007437695693]


In [244]:
scores = model.predict_proba(X_test)
pred = model.predict_classes(X_test)

#### TODO: RECORD ACCURACY IN MODEL RESULTS SPREADSHEET
Try to get accurracy, AUC above 90%!!!
See results for P8 for guidelines

In [245]:
try:
    print("AUC:",roc_auc_score(y_test, scores)) 
except:
    print("AUC undefinied, only 1 class in test data")
    
print("Accuracy:",accuracy_score(y_test, pred))
print(metrics.confusion_matrix(y_test, pred))
print(metrics.classification_report(y_test, pred))

AUC: 0.8553321903822332
Accuracy: 0.650007437695693
[[ 16205  96000]
 [   466 162952]]
              precision    recall  f1-score   support

         0.0       0.97      0.14      0.25    112205
         1.0       0.63      1.00      0.77    163418

   micro avg       0.65      0.65      0.65    275623
   macro avg       0.80      0.57      0.51    275623
weighted avg       0.77      0.65      0.56    275623

